<a href="https://colab.research.google.com/github/AkkiNikumbh/DL-EXPERIMENTS/blob/main/AkashSingh_23CS036_Experiment3_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
import os
import time

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
transform_cifar = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_cifar = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_cifar)

test_cifar = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_cifar)

train_loader_cifar = DataLoader(train_cifar, batch_size=64, shuffle=True)
test_loader_cifar = DataLoader(test_cifar, batch_size=64, shuffle=False)


In [ ]:
class CustomCNN(nn.Module):
    def __init__(self, activation='relu', num_classes=10):
        super(CustomCNN, self).__init__()

        if activation == 'relu':
            act = nn.ReLU()
        elif activation == 'tanh':
            act = nn.Tanh()
        else:
            act = nn.LeakyReLU(0.01)

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            act,
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            act,
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            act,
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(128 * 4 * 4, 256),
            act,
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)


In [ ]:
def initialize_weights(model, init_type):
    for m in model.modules():
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            if init_type == 'xavier':
                nn.init.xavier_uniform_(m.weight)
            elif init_type == 'kaiming':
                nn.init.kaiming_uniform_(m.weight)
            else:
                nn.init.normal_(m.weight, 0, 0.02)


In [ ]:
def train_model(model, loader, optimizer, criterion, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0
        correct = 0
        total = 0

        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        print(f"Epoch {epoch+1} | Loss: {running_loss/len(loader):.4f} | Accuracy: {100*correct/total:.2f}%")


In [ ]:
def evaluate_model(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    return 100 * correct / total


In [ ]:
activations = ['relu', 'tanh', 'leaky']
initializations = ['xavier', 'kaiming', 'random']
optimizers = ['sgd', 'adam', 'rmsprop']

best_acc = 0

for act in activations:
    for init in initializations:
        for opt in optimizers:
            print(f"\nActivation: {act} | Init: {init} | Optimizer: {opt}")

            model = CustomCNN(act, num_classes=10).to(device)
            initialize_weights(model, init)

            criterion = nn.CrossEntropyLoss()

            if opt == 'sgd':
                optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
            elif opt == 'adam':
                optimizer = optim.Adam(model.parameters(), lr=0.001)
            else:
                optimizer = optim.RMSprop(model.parameters(), lr=0.001)

            train_model(model, train_loader_cifar, optimizer, criterion)
            acc = evaluate_model(model, test_loader_cifar)

            print("Test Accuracy:", acc)

            if acc > best_acc:
                best_acc = acc
                torch.save(model.state_dict(), "best_cnn_cifar10.pth")


Activation: relu | Init: xavier | Optimizer: sgd
Epoch 1 | Loss: 1.4756 | Accuracy: 47.53%
Epoch 2 | Loss: 1.1381 | Accuracy: 59.47%
Epoch 3 | Loss: 0.9833 | Accuracy: 65.42%
Epoch 4 | Loss: 0.8798 | Accuracy: 69.02%
Epoch 5 | Loss: 0.8062 | Accuracy: 71.61%
Test Accuracy: 74.51

Activation: relu | Init: xavier | Optimizer: adam
Epoch 1 | Loss: 1.3833 | Accuracy: 51.42%
Epoch 2 | Loss: 0.9872 | Accuracy: 65.12%
Epoch 3 | Loss: 0.8510 | Accuracy: 70.04%
Epoch 4 | Loss: 0.7726 | Accuracy: 73.16%
Epoch 5 | Loss: 0.7118 | Accuracy: 75.10%
Test Accuracy: 76.27

Activation: relu | Init: xavier | Optimizer: rmsprop
Epoch 1 | Loss: 1.4932 | Accuracy: 49.96%
Epoch 2 | Loss: 1.0206 | Accuracy: 64.02%
Epoch 3 | Loss: 0.8793 | Accuracy: 69.06%
Epoch 4 | Loss: 0.7952 | Accuracy: 72.10%
Epoch 5 | Loss: 0.7326 | Accuracy: 74.51%
Test Accuracy: 74.41

Activation: relu | Init: kaiming | Optimizer: sgd
Epoch 1 | Loss: 1.4798 | Accuracy: 47.48%
Epoch 2 | Loss: 1.1230 | Accuracy: 60.14%
Epoch 3 | Loss: 0

In [ ]:
resnet = models.resnet18(pretrained=True)

for param in resnet.parameters():
    param.requires_grad = False

resnet.fc = nn.Linear(resnet.fc.in_features, 10)
resnet = resnet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.fc.parameters(), lr=0.001)

train_model(resnet, train_loader_cifar, optimizer, criterion)
resnet_acc = evaluate_model(resnet, test_loader_cifar)

print("ResNet-18 Accuracy on CIFAR-10:", resnet_acc)


In [ ]:
print("Best Custom CNN Accuracy:", best_acc)
print("ResNet-18 Accuracy:", resnet_acc)